In [59]:
import numpy as np
import pandas as pd
import csv
import os
import re
import io
import json
import pypdf
import langchain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
import tabula
import openai

In [2]:
api_key = json.load(open('./data/creds/gpt_id.json'))['api_key']
openai.api_key = api_key
os.environ['OPENAI_API_KEY'] = api_key

In [10]:
nan_equivalents = [np.NaN, np.nan,
                       'nan', 'NaN', 'Nan', 'NAN', 'na', 'NA',
                       'Unnamed:0',
                       '', '-', ' ', '  ', '   ']

titulo = ' -'

In [11]:
titulo in nan_equivalents

False

In [87]:
def limpieza_texto(texto: str) -> str:
    '''
    Función para limpiar texto de pdfs.
    Cambia saltos de línea, espacios en blanco y caracteres especiales.
    '''
    # Eliminamos espacios en blanco
    #texto = re.sub(' +', ' ', texto)
    # Eliminamos caracteres especiales [REVISAR]
    #texto = re.sub('[^A-Za-z0-9]+', ' ', texto)
    # Eliminamos saltos múltiples de línea
    texto = re.sub(r"\n\s*\n", "\n\n", texto)
    texto = re.sub(r"\n", ";", texto)
    return texto

def tabla_a_texto(tabla):
    '''
    Función para convertir una tabla de pandas en un texto.
    La idea es identificar los nombres de columna e índices correctos y
    a partir de ahí generar un texto que pueda ser procesado por el modelo.
    '''
    tabla = tabla.copy()
    
    # Tamaño mínimo de tabla para que sea válida = 2x2
    if sum(tabla.shape) < 4:
        return ''
    
    # Lista de valores que consideramos NaN:
    nan_equivalents = [np.NaN, np.nan,
                       'nan', 'NaN', 'Nan', 'NAN', 'na', 'NA',
                       'Unnamed:0', 'Unnamed: 0'
                       '', '-', ' ', '  ', '   ']
    
    # Asumimos que el primer elemento es el título salvo si es NaN:
    titulo = tabla.columns[0] if tabla.columns[0] not in nan_equivalents else ''
    
    # Asumimos que la primera columna es el índice y la eliminamos:
    tabla.index = tabla[tabla.columns[0]].values
    tabla.drop(columns=tabla.columns[0], inplace=True)

    # Si las columnas tienen muchos 'Unnamed' suele ser porque hay
    # varias líneas de texto. En ese caso, las juntamos en una sola:
    if sum(['Unnamed' in i for i in tabla.columns]) > 2:
        nueva_columna = [f'{tabla.columns[i]} {tabla.iloc[0,i]}'
                         for i in range(len(tabla.columns))]
        nueva_columna = [i.replace('Unnamed: ','') for i in nueva_columna]
        tabla.columns = nueva_columna

    
    # Eliminamos las filas y columnas que no tienen datos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    
    # Check si las columnas son años:
    col_años = False
    years_txt=[str(i) for i in range(2015,2022)]
    years_int=[i for i in range(2015,2022)]
    years = set(years_txt+years_int)
    cruce = set(tabla.columns).intersection(set(years))
    if len(cruce) > 0: col_años=True
    
    # Si no son años las columnas, buscamos filas que sean años:
    contexto = None
    if not col_años:
        for i in tabla.iterrows():
            #print(i[1].values)
            try:
                cruce = set(i[1].values).intersection(set(years))
            except:
                cruce=[]
            if len(cruce)>0: # Si encontramos una fila con años:
                # Asignamos los años a las columnas:
                tabla.columns = i[1].values
                try: 
                    contexto = i[1].name
                except:
                    contexto = None
                # Drop de la fila:
                tabla.drop(i[0], inplace=True)
                break
    # Los procesos anteriores pueden haber dejado filas vacías, las eliminamos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    # Pasamos a texto:
    texto = ''
    for i in tabla.items():
        txt = [f' {titulo} + {i[0]} + {x[0]} = {x[1]}; '
            for x in list(i[1].items())]
        add= ''.join(txt)
        if contexto:
            txt = [f' {titulo} + {contexto} + {i[0]} + {x[0]} = {x[1]}; '
                for x in list(i[1].items())]
            add = ''.join(txt)
        add = add.replace('  ',' ').replace('\n','; ').replace('  ','')
        texto += f';  Tabla={titulo}: {add}'
    return texto

def extract_text_from_pdf(pdf_path: str) -> list:
    '''
    Función para extraer texto de un pdf y limpiarlo.
    Devuelve una lista de str, cada una es una página del pdf.
    '''
    # Abrimos el pdf
    with open(pdf_path, 'rb') as f:
        pdf = pypdf.PdfReader(f)
        # Obtenemos el número de páginas
        num_pags = len(pdf.pages)
        count = 0
        text = []
        # Iteramos sobre las páginas
        for pag in pdf.pages:
            count +=1
            texto_pagina = pag.extract_text()
            tablas = tabula.read_pdf(pdf_path, pages=count)
            for tabla in tablas:
                texto_pagina += f'; {tabla_a_texto(tabla=tabla)}; '
            texto_pagina = limpieza_texto(texto_pagina)
            text.append(texto_pagina)
    return text

In [88]:
# Test con un pdf
resultado = extract_text_from_pdf('./data/rovi-sostenibilidad-2020.pdf')

In [89]:
# Guardamos el resultado en un csv con 1 fila por página
with open('lista_test.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerows([[str(i)] for i in resultado])

In [90]:
splitter = langchain.text_splitter.RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function = len)

x = splitter.create_documents(resultado, metadatas=[{'pag': i} for i in list(range(len(resultado)))])

In [69]:
x[0].page_content

'REPORT;20;20'

In [91]:
loader = CSVLoader(file_path='lista_test.csv')
#data = loader.load()

In [29]:
data

[Document(page_content='REPORT;20;20: CONTENTS;2', lookup_str='', metadata={'source': 'lista_test.csv', 'row': 0}, lookup_index=0),
 Document(page_content='REPORT;20;20: 1. ABOUT ROVI;2. RESPONSIBLE AND SUSTAINABLE MANAGEMENT;4. HUMAN RESOURCES;5. SUPPLIERS;6. SOCIETY;8. ENVIRONMENT;9. SUPPLEMENTARY INFORMATION7. SHAREHOLDERS3. CUSTOMERS, PATIENTS AND PROFESSIONALS1.1. Group profile;2.1. Statement by the Chief Executive Officer;4.1. Management principles;5.1. Management principles;6.1. Management principles;8.1. Management principles ;9.1. About this Report3.1. Management principles1.2. Business model;2.2. CSR strategy and objectives;4.2. Balance of the year and future challenges;5.2. Balance of the year and future challenges ;6.2. Balance of the year;8.2. Consumption and waste;9.2. GRI Content Index3.2. Balance of the year and future challenges1.3. 1.3. World presence;2.3. Our stakeholders;4.3. Equal opportunities and work-life balance3.3. Innovation2.4. Materiality analysis;4.4. Attr

In [92]:
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


In [96]:
index.query_with_sources('There is a table talking about CO2 emissions. How much CO2 did they emit?')

{'question': 'There is a table talking about CO2 emissions. How much CO2 did they emit?',
 'answer': ' The CO2 emissions in 2020 were 1,262 tonnes of Scope 1 CO2 emitted.\n',
 'sources': 'lista_test.csv'}

In [65]:
+805+1494+2663+836+468+1399+2280

9945

In [28]:
index.vectorstore.

In [130]:
with open('./data/rovi-sostenibilidad-2020.pdf', 'rb') as f:
    pdf = pypdf.PdfReader(f)
    # Obtenemos el número de páginas
    num_pags = len(pdf.pages)
    count = 0
    text = []
    # Iteramos sobre las páginas
    for pag in pdf.pages:
    #while count < num_pags:
        #pag = pdf.pages[count]
        count +=1
        texto_pagina = pag.extract_text()
        tablas = tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=count)
        for tabla in tablas:
            texto_pagina += tabla_a_texto(tabla=tabla)
        # texto_pagina = limpieza_texto(texto_pagina)
        text.append(texto_pagina)

In [5]:
with open('./data/rovi-sostenibilidad-2020.pdf', 'rb') as f:
    pdf = pypdf.PdfReader(f)

In [139]:
tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=count)

[]

In [142]:
test

';  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2020.0 2-Datos: [Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: 2020.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2019.0 2-Datos: [Total revenue 382.50; EBITDA 60.90; Net financial debt 15.90; Employees1.31; Name: 2019.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2018.0 2-Datos: [Total revenue 304.800; EBITDA 29.500; Net financial debt-62.800; Employees1.224; Name: 2018.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2017.0 2-Datos: [Total revenue 277.400; EBITDA 29.900; Net financial debt 1.100; Employees1.191; Name: 2017.0, dtype: float64]'

In [144]:
import langchain
langchain.docstore.document.Document(page_content = test)

Document(page_content=';  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2020.0 2-Datos: [Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: 2020.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2019.0 2-Datos: [Total revenue 382.50; EBITDA 60.90; Net financial debt 15.90; Employees1.31; Name: 2019.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2018.0 2-Datos: [Total revenue 304.800; EBITDA 29.500; Net financial debt-62.800; Employees1.224; Name: 2018.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2017.0 2-Datos: [Total revenue 277.400; EBITDA 29.900; Net financial debt 1.100; Employees1.191; Name: 2017.0, dtype: float64]', lookup_str='', metadata={}, lookup_index=0)

In [135]:
tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=1)[0]

IndexError: list index out of range

In [19]:
tabla_ej = tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=91)[0]
tabla_ej

,Unnamed: 0,Unnamed: 1,Madrid,Alcalá de,Unnamed: 2,Unnamed: 3,Madrid.1,Alcalá de.1,Unnamed: 4,Unnamed: 5,Mad,Unnamed: 6,Unnamed: 7
0,NaN,Granada,& SSRR,Henares,Distr.,Granada,& SSRR,Henares,Distr.,Gr,y SSRR,AH,Distr.
1,Tonnes of Scope 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CO2 emitted,805,"1,494","2,663",836,468,"1,399","2,880","1,262",72%,7%,-8%,-34%
3,Tonnes of Scope,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2 CO2 emitted,0,0,0,102,"1,101","2,245","2,565",179,-100%,-100%,-100%,-43%
5,Tonnes of Scope,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2 CO2 avoided (*),"1,193","2,198","2,999",96,0,0,0,0,-,-,-,-
7,Tonnes CO2/ /,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,million units.,0.004,22.58,104.85,42.30,0.003,27.54,137.86,71.42,38%,-18%,-24%,-41%


In [75]:
# Si las columnas tienen muchos 'Unnamed' suele ser porque hay
# varias líneas de texto. En ese caso, las juntamos en una sola:
if sum(['Unnamed' in i for i in tabla_ej.columns]) > 2:
    print('hola')
    nueva_columna = [f'{tabla_ej.columns[i]} {tabla_ej.iloc[0,i]}' for i in range(len(tabla_ej.columns))]
    nueva_columna = [i.replace('Unnamed: ','') for i in nueva_columna]
    print(nueva_columna)

hola
['0 nan', '1 Granada', 'Madrid & SSRR', 'Alcalá de Henares', '2 Distr.', '3 Granada', 'Madrid.1 & SSRR', 'Alcalá de.1 Henares', '4 Distr.', '5 Gr', 'Mad y SSRR', '6 AH', '7 Distr.']


In [85]:
def tabla_a_texto(tabla):
    '''
    Función para convertir una tabla de pandas en un texto.
    La idea es identificar los nombres de columna e índices correctos y
    a partir de ahí generar un texto que pueda ser procesado por el modelo.
    '''
    tabla = tabla.copy()
    
    # Tamaño mínimo de tabla para que sea válida = 2x2
    if sum(tabla.shape) < 4:
        return ''
    
    # Lista de valores que consideramos NaN:
    nan_equivalents = [np.NaN, np.nan,
                       'nan', 'NaN', 'Nan', 'NAN', 'na', 'NA',
                       'Unnamed:0', 'Unnamed: 0'
                       '', '-', ' ', '  ', '   ']
    
    # Asumimos que el primer elemento es el título salvo si es NaN:
    titulo = tabla.columns[0] if tabla.columns[0] not in nan_equivalents else ''
    
    # Asumimos que la primera columna es el índice y la eliminamos:
    tabla.index = tabla[tabla.columns[0]].values
    tabla.drop(columns=tabla.columns[0], inplace=True)

    # Si las columnas tienen muchos 'Unnamed' suele ser porque hay
    # varias líneas de texto. En ese caso, las juntamos en una sola:
    if sum(['Unnamed' in i for i in tabla.columns]) > 2:
        nueva_columna = [f'{tabla.columns[i]} {tabla.iloc[0,i]}'
                         for i in range(len(tabla.columns))]
        nueva_columna = [i.replace('Unnamed: ','') for i in nueva_columna]
        tabla.columns = nueva_columna

    
    # Eliminamos las filas y columnas que no tienen datos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    
    # Check si las columnas son años:
    col_años = False
    years_txt=[str(i) for i in range(2015,2022)]
    years_int=[i for i in range(2015,2022)]
    years = set(years_txt+years_int)
    cruce = set(tabla.columns).intersection(set(years))
    if len(cruce) > 0: col_años=True
    
    # Si no son años las columnas, buscamos filas que sean años:
    contexto = None
    if not col_años:
        for i in tabla.iterrows():
            #print(i[1].values)
            try:
                cruce = set(i[1].values).intersection(set(years))
            except:
                cruce=[]
            if len(cruce)>0: # Si encontramos una fila con años:
                # Asignamos los años a las columnas:
                tabla.columns = i[1].values
                try: 
                    contexto = i[1].name
                except:
                    contexto = None
                # Drop de la fila:
                tabla.drop(i[0], inplace=True)
                break
    # Los procesos anteriores pueden haber dejado filas vacías, las eliminamos:
    tabla.replace(nan_equivalents, np.nan, inplace=True)
    tabla.dropna(axis=0, how='all', inplace=True)
    tabla.dropna(axis=1, how='all', inplace=True)
    # Pasamos a texto:
    texto = ''
    for i in tabla.items():
        txt = [f' {titulo} + {i[0]} + {x[0]} = {x[1]}; '
            for x in list(i[1].items())]
        add= ''.join(txt)
        if contexto:
            txt = [f' {titulo} + {contexto} + {i[0]} + {x[0]} = {x[1]}; '
                for x in list(i[1].items())]
            add = ''.join(txt)
        add = add.replace('  ',' ').replace('\n','; ').replace('  ','')
        texto += f';  Tabla={titulo}: {add}'
    return texto

In [86]:
tabla_a_texto(tabla_ej)

';  Tabla=:  + 1 Granada + nan = Granada;+ 1 Granada + CO2 emitted = 805;+ 1 Granada + 2 CO2 emitted = 0;+ 1 Granada + 2 CO2 avoided (*) = 1,193;+ 1 Granada + million units. = 0.004; ;  Tabla=:  + Madrid & SSRR + nan = & SSRR;+ Madrid & SSRR + CO2 emitted = 1,494;+ Madrid & SSRR + 2 CO2 emitted = 0;+ Madrid & SSRR + 2 CO2 avoided (*) = 2,198;+ Madrid & SSRR + million units. = 22.58; ;  Tabla=:  + Alcalá de Henares + nan = Henares;+ Alcalá de Henares + CO2 emitted = 2,663;+ Alcalá de Henares + 2 CO2 emitted = 0;+ Alcalá de Henares + 2 CO2 avoided (*) = 2,999;+ Alcalá de Henares + million units. = 104.85; ;  Tabla=:  + 2 Distr. + nan = Distr.;+ 2 Distr. + CO2 emitted = 836;+ 2 Distr. + 2 CO2 emitted = 102;+ 2 Distr. + 2 CO2 avoided (*) = 96;+ 2 Distr. + million units. = 42.30; ;  Tabla=:  + 3 Granada + nan = Granada;+ 3 Granada + CO2 emitted = 468;+ 3 Granada + 2 CO2 emitted = 1,101;+ 3 Granada + 2 CO2 avoided (*) = 0;+ 3 Granada + million units. = 0.003; ;  Tabla=:  + Madrid.1 & SSRR + 

In [50]:
tabla = tabla_ej.copy()
    
# Tamaño mínimo de tabla para que sea válida = 2x2
if sum(tabla.shape) < 4:
    None

# Lista de valores que consideramos NaN:
nan_equivalents = [np.NaN, np.nan,
                    'nan', 'NaN', 'Nan', 'NAN', 'na', 'NA',
                    'Unnamed:0',
                    '', '-', ' ', '  ', '   ']

# Asumimos que el primer elemento es el título salvo si es NaN:
titulo = tabla.columns[0] if tabla.columns[0] not in nan_equivalents else ''

# Asumimos que la primera columna es el índice y la eliminamos:
tabla.index = tabla[tabla.columns[0]].values
tabla.drop(columns=tabla.columns[0], inplace=True)

# Eliminamos las filas y columnas que no tienen datos:
tabla.replace(nan_equivalents, np.nan, inplace=True)
tabla.dropna(axis=0, how='all', inplace=True)
tabla.dropna(axis=1, how='all', inplace=True)

# Check si las columnas son años:
col_años = False
years_txt=[str(i) for i in range(2015,2022)]
years_int=[i for i in range(2015,2022)]
years = set(years_txt+years_int)
cruce = set(tabla.columns).intersection(set(years))
if len(cruce) > 0: col_años=True

# Si no son años las columnas, buscamos filas que sean años:
contexto = None
if not col_años:
    for i in tabla.iterrows():
        #print(i[1].values)
        try:
            cruce = set(i[1].values).intersection(set(years))
        except:
            cruce=[]
        if len(cruce)>0: # Si encontramos una fila con años:
            # Asignamos los años a las columnas:
            tabla.columns = i[1].values
            try: 
                contexto = i[1].name
            except:
                contexto = None
            # Drop de la fila:
            tabla.drop(i[0], inplace=True)
            break
# Los procesos anteriores pueden haber dejado filas vacías, las eliminamos:
tabla.replace(nan_equivalents, np.nan, inplace=True)
tabla.dropna(axis=0, how='all', inplace=True)
tabla.dropna(axis=1, how='all', inplace=True)
# Pasamos a texto:



In [51]:
texto = ''
for i in tabla.items():
    txt = [f'{titulo} + {i[0]} + {x[0]} = {x[1]}; '
           for x in list(i[1].items())]
    add= ''.join(txt)
    if contexto:
        txt = [f'{titulo} + {contexto} + {i[0]} + {x[0]} = {x[1]}; '
               for x in list(i[1].items())]
        add = ''.join(txt)
    add = add.replace('  ',' ').replace('\n','; ').replace('  ','')
    texto += f';  Tabla={titulo}: {add}'

In [52]:
texto

';  Tabla=Unnamed: 0: Unnamed: 0 + Unnamed: 1 + nan = Granada; Unnamed: 0 + Unnamed: 1 + CO2 emitted = 805; Unnamed: 0 + Unnamed: 1 + 2 CO2 emitted = 0; Unnamed: 0 + Unnamed: 1 + 2 CO2 avoided (*) = 1,193; Unnamed: 0 + Unnamed: 1 + million units. = 0.004; ;  Tabla=Unnamed: 0: Unnamed: 0 + Madrid + nan = & SSRR; Unnamed: 0 + Madrid + CO2 emitted = 1,494; Unnamed: 0 + Madrid + 2 CO2 emitted = 0; Unnamed: 0 + Madrid + 2 CO2 avoided (*) = 2,198; Unnamed: 0 + Madrid + million units. = 22.58; ;  Tabla=Unnamed: 0: Unnamed: 0 + Alcalá de + nan = Henares; Unnamed: 0 + Alcalá de + CO2 emitted = 2,663; Unnamed: 0 + Alcalá de + 2 CO2 emitted = 0; Unnamed: 0 + Alcalá de + 2 CO2 avoided (*) = 2,999; Unnamed: 0 + Alcalá de + million units. = 104.85; ;  Tabla=Unnamed: 0: Unnamed: 0 + Unnamed: 2 + nan = Distr.; Unnamed: 0 + Unnamed: 2 + CO2 emitted = 836; Unnamed: 0 + Unnamed: 2 + 2 CO2 emitted = 102; Unnamed: 0 + Unnamed: 2 + 2 CO2 avoided (*) = 96; Unnamed: 0 + Unnamed: 2 + million units. = 42.30; ; 

In [49]:
''.join([f'{i[0]} + {x[0]} = {x[1]}; ' for x in list(i[1].items())])

'Unnamed: 1 + nan = Granada; Unnamed: 1 + CO2 emitted = 805; Unnamed: 1 + 2 CO2 emitted = 0; Unnamed: 1 + 2 CO2 avoided (*) = 1,193; Unnamed: 1 + million units. = 0.004; '

In [43]:
add

'Unnamed: 1 + nan = GranadaUnnamed: 1 + CO2 emitted = 805Unnamed: 1 + 2 CO2 emitted = 0Unnamed: 1 + 2 CO2 avoided (*) = 1,193Unnamed: 1 + million units. = 0.004'

In [35]:
list(i[1].items())

[(nan, 'Granada'),
 ('CO2 emitted', '805'),
 ('2 CO2 emitted', '0'),
 ('2 CO2 avoided (*)', '1,193'),
 ('million units.', '0.004')]

In [128]:
test = tabla_a_texto(tabla_ej)

In [129]:
test

';  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2020.0 2-Datos: [Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: 2020.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2019.0 2-Datos: [Total revenue 382.50; EBITDA 60.90; Net financial debt 15.90; Employees1.31; Name: 2019.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2018.0 2-Datos: [Total revenue 304.800; EBITDA 29.500; Net financial debt-62.800; Employees1.224; Name: 2018.0, dtype: float64];  KEY FIGURES: A-Contexto: (million euros) B-Contenido:1-Columna: 2017.0 2-Datos: [Total revenue 277.400; EBITDA 29.900; Net financial debt 1.100; Employees1.191; Name: 2017.0, dtype: float64]'

In [101]:
tabla_a_texto(tabla_ej)

                     2020.0  2019.0   2018.0   2017.0
KEY FIGURES                                          
Total revenue       421.100  382.50  304.800  277.400
EBITDA               94.200   60.90   29.500   29.900
Net financial debt   19.800   15.90  -62.800    1.100
Employees             1.419    1.31    1.224    1.191
                     2020.0  2019.0   2018.0   2017.0
KEY FIGURES                                          
Total revenue       421.100  382.50  304.800  277.400
EBITDA               94.200   60.90   29.500   29.900
Net financial debt   19.800   15.90  -62.800    1.100
Employees             1.419    1.31    1.224    1.191


'KEY FIGURES;  KEY FIGURES: 1-Columna: 2020.0 2-Datos: [KEY FIGURES; Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: 2020.0, dtype: float64];  KEY FIGURES: 1-Columna: 2019.0 2-Datos: [KEY FIGURES; Total revenue 382.50; EBITDA 60.90; Net financial debt 15.90; Employees1.31; Name: 2019.0, dtype: float64];  KEY FIGURES: 1-Columna: 2018.0 2-Datos: [KEY FIGURES; Total revenue 304.800; EBITDA 29.500; Net financial debt-62.800; Employees1.224; Name: 2018.0, dtype: float64];  KEY FIGURES: 1-Columna: 2017.0 2-Datos: [KEY FIGURES; Total revenue 277.400; EBITDA 29.900; Net financial debt 1.100; Employees1.191; Name: 2017.0, dtype: float64]'

In [14]:
tabla_ej.index = tabla_ej[tabla_ej.columns[0]]
tabla_ej.drop(columns=tabla_ej.columns[0], inplace=True)

In [49]:
f'1-Columna: {i[0]} 2-Datos: [{i[1]}]'.replace('  ',' ').replace('\n','; ').replace('  ','')

'1-Columna: Unnamed: 0 2-Datos: [KEY FIGURES; (million euros)2020.000; Total revenue 421.100; EBITDA 94.200; Net financial debt 19.800; Employees1.419; Name: Unnamed: 0, dtype: float64]'

In [34]:
for i in tabla_ej.iteritems():
    print(i)
    break

('Unnamed: 0', KEY FIGURES
(million euros)       2020.000
Total revenue          421.100
EBITDA                  94.200
Net financial debt      19.800
Employees                1.419
Name: Unnamed: 0, dtype: float64)


C:\Users\Faji\AppData\Local\Temp\ipykernel_2796\1429811112.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i in tabla_ej.iteritems():


In [38]:
str(i)

"('Unnamed: 0', KEY FIGURES\n(million euros)       2020.000\nTotal revenue          421.100\nEBITDA                  94.200\nNet financial debt      19.800\nEmployees                1.419\nName: Unnamed: 0, dtype: float64)"

In [20]:
tabla_ej

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,102-7
KEY FIGURES,,,,,
(million euros),2020.000,2019.00,2018.000,2017.000,NaN
Total revenue,421.100,382.50,304.800,277.400,NaN
EBITDA,94.200,60.90,29.500,29.900,NaN
Net financial debt,19.800,15.90,-62.800,1.100,NaN
Employees,1.419,1.31,1.224,1.191,NaN


In [151]:
def limpieza_passages(texto):
  '''
  Dado un texto realiza una limpieza caracteres especiales, transformar a minusculas y eleiminar urls
  Args:
        texto (str): texto a limpiar

    Returns:
        text(str): texto limpio
  '''
  texto=texto.replace("\n"," ")
  texto=texto.replace("\t"," ")
  texto=texto.replace("\\r"," ")
  texto=texto.replace("\r"," ")
  texto=texto.replace(" - ","")
  texto=texto.replace("- ","")
  texto=texto.replace("-","")
  texto=texto.replace(" -","")
  texto=texto.replace(">"," ")
  texto=texto.lower()
  texto = re.sub(r'^https?:\/\/.*[\r\n]*', '', texto, flags=re.MULTILINE)
 
  return texto

def columnas_son_años(columnas):
  '''
  Dado una lista de columnas determina si son años 
  Args:
        columnas list(str): nombre de las columnas de una tabla
       
  Return:
      Bool: indica si son años o no
  '''
  años=['2019','2020','2021','2018','2017']
  años = set(años)
  columnas = set(columnas)
  son=False
  interseccion= set(columnas).intersection(años)
  interseccion = list(interseccion)

  
  if len(interseccion)>1:
    son=True
  return son 

def tabla_to_lines(df,año_actual):
  '''
  Extrae la informacion de una tabla a un texto. Ademas si las columnas son años, solo extrae la informacion del año que estamos estudiando
  Args:
        df Dataframe: tabla en formato dataframe de la que queremos extraer la informacion.
        año_actual str: año del que hacemos el estudio
       
  Return:
      txt: con la informacion de la tabla
  '''
  columnas=list(df.columns)
  num_columnas=df.shape[1]
  num_filas=df.shape[0]
  if num_filas>0 and num_columnas>0:
    titulo=titulo=columnas[0]
    filas=list(df.iloc[0:,0])
    texto=''
    if columnas_son_años(columnas[1:])==False:
      for i in range(0,num_filas):
        
        for j in range(1,num_columnas):
          
          dato=df.iloc[i,j]
          if str(dato)!='Nan' and str(dato)!='na' and str(dato)!='NaN' and str(dato)!='nan':
            if str(columnas[j])[0:8]!='Unnamed:':
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[j])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(columnas[j])+'             '
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[j])+'             '
                else: 
                  texto=texto+' '+str(dato)+' '+str(columnas[j])+'             '
              
            else:
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+'             '
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+'             '
                else: 
                  texto=texto+' '+str(dato)+'                  '
    else:
      indice_actual=columnas.index(año_actual)
      for i in range(0,num_filas):
        
        
          
          dato=df.iloc[i,indice_actual]
          if str(dato)!='Nan' and str(dato)!='na' and str(dato)!='NaN' and str(dato)!='nan':
            
              if str(titulo) !='Unnamed: 0':
                if str(filas[i])!='nan':

                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[indice_actual])+'             '
                else: 
                  texto=texto+' '+str(titulo)+' '+str(dato)+' '+str(columnas[indice_actual])+'             '
                
              else:
                if str(filas[i])!='nan':
                  texto=texto+' '+str(dato)+' '+str(filas[i])+' '+str(columnas[indice_actual])+'             '
                else: 
                  texto=texto+' '+str(dato)+' '+str(columnas[indice_actual])+'                  '
             
    return limpieza_passages(texto)

In [22]:
tabla_to_lines(tabula.read_pdf('./data/rovi-sostenibilidad-2020.pdf', pages=14)[0],'2020')

' key figures 2020.0 (million euros)              key figures 2019.0 (million euros)              key figures 2018.0 (million euros)              key figures 2017.0 (million euros)              key figures 421.1 total revenue              key figures 382.5 total revenue              key figures 304.8 total revenue              key figures 277.4 total revenue              key figures 94.2 ebitda              key figures 60.9 ebitda              key figures 29.5 ebitda              key figures 29.9 ebitda              key figures 19.8 net financial debt              key figures 15.9 net financial debt              key figures 62.8 net financial debt              key figures 1.1 net financial debt              key figures 1.419 employees              key figures 1.31 employees              key figures 1.224 employees              key figures 1.191 employees             '

In [26]:
columnas_son_años(tabla_ej.columns)

False

In [37]:
tabla_ej.iloc[0,:].values

array(['(million euros)', 2020.0, 2019.0, 2018.0, 2017.0, nan],
      dtype=object)

In [39]:
set(tabla_ej.iloc[0,:].values).intersection(set([2019,2020,2021,2018,2017]))

{2017, 2018, 2019, 2020}

In [41]:
int(años)

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [ ]:
def filas_son_años2(df):
  '''
  xx
  '''
  years_txt=['2019','2020','2021','2018','2017']
  years_int=[2019,2020,2021,2018,2017]
  years = set(years_txt+years_int)
  columnas = set(columnas)
  son=False
  interseccion= set(columnas).intersection(years)
  interseccion = list(interseccion)
  
  if len(interseccion)>1:
    son=True
    return son
  else:

In [58]:
years_txt=['2019','2020','2021','2018','2017']
years_int=[2019,2020,2021,2018,2017]
years = set(years_txt+years_int)

In [61]:
for i in tabla_ej.iterrows():
    #print(i[1].values)
    try:
        cruce = set(i[1].values).intersection(set(years))
        if len(cruce)>0:
            return i[0]
    break

{2017.0, 2018.0, 2019.0, 2020.0}


In [71]:
tabla_ej.iloc[:,0]

0       (million euros)
1         Total revenue
2                EBITDA
3    Net financial debt
4             Employees
Name: KEY FIGURES, dtype: object

In [72]:
tabla_ej.set_index(tabla_ej.iloc[:,0].values)

,KEY FIGURES,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,102-7
(million euros),(million euros),2020.000,2019.00,2018.000,2017.000,NaN
Total revenue,Total revenue,421.100,382.50,304.800,277.400,NaN
EBITDA,EBITDA,94.200,60.90,29.500,29.900,NaN
Net financial debt,Net financial debt,19.800,15.90,-62.800,1.100,NaN
Employees,Employees,1.419,1.31,1.224,1.191,NaN
